In [1]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from dataclasses import dataclass
from tqdm import tqdm
import multiprocessing

In [6]:

#(Hyperparameters)
@dataclass
class ModelConfig:
    # File Paths
    RATINGS_PATH: str = '/content/drive/MyDrive/dataset/ratings.csv'
    MOVIES_PATH: str = '/content/drive/MyDrive/dataset/movies_encoded.csv' # Optional
    MODEL_SAVE_PATH: str = 'movie_recommender_model.pth'

    # Model Architecture
    EMBEDDING_DIM: int = 64  # Increased slightly for better capacity

    # Training Hyperparameters
    BATCH_SIZE: int = 2048   # Increased from 128 for better GPU throughput
    EPOCHS: int = 10         # 20 might be overkill for this simple architecture
    LEARNING_RATE: float = 0.01
    WEIGHT_DECAY: float = 1e-5
    TEST_SIZE: float = 0.2

    # Hardware / System
    DEVICE: str = 'cuda' if torch.cuda.is_available() else 'cpu'
    # Auto-detect CPU cores for data loading, clamp to 8 to avoid overhead
    NUM_WORKERS: int = min(8, multiprocessing.cpu_count())
    PIN_MEMORY: bool = True if torch.cuda.is_available() else False

config = ModelConfig()

In [3]:

# DATASET & MODEL

class MovieRatingsDataset(Dataset):
    def __init__(self, user_ids, item_ids, ratings):
        self.user_ids = torch.LongTensor(user_ids.values)
        self.item_ids = torch.LongTensor(item_ids.values)
        self.ratings = torch.FloatTensor(ratings.values)

    def __len__(self):
        return len(self.ratings)

    def __getitem__(self, idx):
        return self.user_ids[idx], self.item_ids[idx], self.ratings[idx]

class MatrixFactorizationWithBias(nn.Module):
    def __init__(self, num_users, num_items, embedding_dim=50):
        super(MatrixFactorizationWithBias, self).__init__()
        self.user_embeddings = nn.Embedding(num_users, embedding_dim)
        self.item_embeddings = nn.Embedding(num_items, embedding_dim)
        self.user_bias = nn.Embedding(num_users, 1)
        self.item_bias = nn.Embedding(num_items, 1)
        self.global_bias = nn.Parameter(torch.zeros(1))

        # Init weights
        self.user_embeddings.weight.data.uniform_(0, 0.05)
        self.item_embeddings.weight.data.uniform_(0, 0.05)
        self.user_bias.weight.data.uniform_(-0.01, 0.01)
        self.item_bias.weight.data.uniform_(-0.01, 0.01)

    def forward(self, user_ids, item_ids):
        user_vecs = self.user_embeddings(user_ids)
        item_vecs = self.item_embeddings(item_ids)
        dot_product = (user_vecs * item_vecs).sum(dim=1)

        user_b = self.user_bias(user_ids).squeeze()
        item_b = self.item_bias(item_ids).squeeze()

        return dot_product + user_b + item_b + self.global_bias

In [4]:

# RECOMMENDER SYSTEM

class MovieRecommender:
    def __init__(self, config: ModelConfig):
        self.cfg = config
        print(f"🚀 Initializing on device: {self.cfg.DEVICE}")

        # Load Data
        print(f"📂 Loading data from {self.cfg.RATINGS_PATH}...")
        self.ratings_df = pd.read_csv(self.cfg.RATINGS_PATH)

        # Load Metadata (Optional)
        self.movies_df = None
        if self.cfg.MOVIES_PATH and os.path.exists(self.cfg.MOVIES_PATH):
            self.movies_df = pd.read_csv(self.cfg.MOVIES_PATH)
            print(f"   ✓ Loaded metadata for {len(self.movies_df)} movies")

        # Mappings
        self._create_mappings()

        # Model placeholders
        self.model = None

    def _create_mappings(self):
        unique_users = sorted(self.ratings_df['userId'].unique())
        unique_items = sorted(self.ratings_df['movieId'].unique())

        self.user_to_idx = {user: idx for idx, user in enumerate(unique_users)}
        self.idx_to_user = {idx: user for user, idx in self.user_to_idx.items()}

        self.item_to_idx = {item: idx for idx, item in enumerate(unique_items)}
        self.idx_to_item = {idx: item for item, idx in self.item_to_idx.items()}

        # Map IDs in DataFrame efficiently
        self.ratings_df['user_idx'] = self.ratings_df['userId'].map(self.user_to_idx)
        self.ratings_df['item_idx'] = self.ratings_df['movieId'].map(self.item_to_idx)

        self.num_users = len(unique_users)
        self.num_items = len(unique_items)
        print(f"   ✓ Users: {self.num_users:,} | Items: {self.num_items:,} | Ratings: {len(self.ratings_df):,}")

    def prepare_data(self):
        train_df, test_df = train_test_split(
            self.ratings_df, test_size=self.cfg.TEST_SIZE, random_state=42
        )

        self.mean_rating = train_df['rating'].mean()

        # Datasets
        train_dataset = MovieRatingsDataset(train_df['user_idx'], train_df['item_idx'], train_df['rating'])
        test_dataset = MovieRatingsDataset(test_df['user_idx'], test_df['item_idx'], test_df['rating'])

        # DataLoaders (Optimized with num_workers and pin_memory)
        self.train_loader = DataLoader(
            train_dataset, batch_size=self.cfg.BATCH_SIZE, shuffle=True,
            num_workers=self.cfg.NUM_WORKERS, pin_memory=self.cfg.PIN_MEMORY
        )
        self.test_loader = DataLoader(
            test_dataset, batch_size=self.cfg.BATCH_SIZE, shuffle=False,
            num_workers=self.cfg.NUM_WORKERS, pin_memory=self.cfg.PIN_MEMORY
        )
        print(f"   ✓ Data split & Loaders ready (Batch Size: {self.cfg.BATCH_SIZE})")

    def train(self):
        self.model = MatrixFactorizationWithBias(
            self.num_users, self.num_items, self.cfg.EMBEDDING_DIM
        ).to(self.cfg.DEVICE)

        self.model.global_bias.data.fill_(self.mean_rating)

        criterion = nn.MSELoss()
        optimizer = optim.Adam(self.model.parameters(), lr=self.cfg.LEARNING_RATE, weight_decay=self.cfg.WEIGHT_DECAY)
        scaler = torch.amp.GradScaler(enabled=(self.cfg.DEVICE == 'cuda')) # Mixed Precision

        best_loss = float('inf')

        print("\n🔄 Starting Training...")
        for epoch in range(self.cfg.EPOCHS):
            # Training Phase
            self.model.train()
            train_loss = 0.0
            pbar = tqdm(self.train_loader, desc=f"Epoch {epoch+1}/{self.cfg.EPOCHS} [Train]")

            for user_ids, item_ids, ratings in pbar:
                user_ids, item_ids, ratings = user_ids.to(self.cfg.DEVICE), item_ids.to(self.cfg.DEVICE), ratings.to(self.cfg.DEVICE)

                optimizer.zero_grad()

                # Mixed Precision Forward/Backward
                with torch.amp.autocast(device_type=self.cfg.DEVICE if self.cfg.DEVICE != 'mps' else 'cpu', enabled=(self.cfg.DEVICE == 'cuda')):
                    predictions = self.model(user_ids, item_ids)
                    loss = criterion(predictions, ratings)

                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scaler.update()

                train_loss += loss.item()
                pbar.set_postfix({'loss': loss.item()})

            avg_train_loss = train_loss / len(self.train_loader)

            # Validation Phase
            self.model.eval()
            test_loss = 0.0
            with torch.no_grad():
                for user_ids, item_ids, ratings in self.test_loader:
                    user_ids, item_ids, ratings = user_ids.to(self.cfg.DEVICE), item_ids.to(self.cfg.DEVICE), ratings.to(self.cfg.DEVICE)
                    predictions = self.model(user_ids, item_ids)
                    loss = criterion(predictions, ratings)
                    test_loss += loss.item()

            avg_test_loss = test_loss / len(self.test_loader)
            rmse = np.sqrt(avg_test_loss)

            print(f"   Done. Train Loss: {avg_train_loss:.4f} | Test Loss: {avg_test_loss:.4f} | RMSE: {rmse:.4f}")

            if avg_test_loss < best_loss:
                best_loss = avg_test_loss
                self.save_model()

    def recommend_top_k(self, user_id, k=3):
        """
        Optimized vectorized inference.
        Calculates scores for ALL unrated items in a single GPU batch instead of a loop.
        """
        if user_id not in self.user_to_idx:
            print(f"❌ User {user_id} not found.")
            return None

        user_idx = self.user_to_idx[user_id]

        # Find unrated items
        rated_items = set(self.ratings_df[self.ratings_df['user_idx'] == user_idx]['item_idx'].values)
        all_items = set(self.item_to_idx.values())
        unrated_items = list(all_items - rated_items)

        if not unrated_items:
            return None

        # Prepare tensors for batch prediction
        user_tensor = torch.tensor([user_idx] * len(unrated_items), dtype=torch.long, device=self.cfg.DEVICE)
        item_tensor = torch.tensor(unrated_items, dtype=torch.long, device=self.cfg.DEVICE)

        self.model.eval()
        with torch.no_grad():
            predictions = self.model(user_tensor, item_tensor)

        # Get Top K indices
        top_k_vals, top_k_indices = torch.topk(predictions, k)

        results = []
        for i in range(len(top_k_indices)):
            item_idx = unrated_items[top_k_indices[i].item()]
            original_item_id = self.idx_to_item[item_idx]
            pred_rating = top_k_vals[i].item()

            res = {'movieId': original_item_id, 'predicted_rating': pred_rating}

            # Attach Metadata if available
            if self.movies_df is not None:
                meta = self.movies_df[self.movies_df['movieId'] == original_item_id]
                if not meta.empty:
                    res['title'] = meta.iloc[0].get('title', 'Unknown')
                    res['genres'] = meta.iloc[0].get('genres', 'Unknown')

            results.append(res)

        return pd.DataFrame(results)

    def save_model(self):
        torch.save(self.model.state_dict(), self.cfg.MODEL_SAVE_PATH)
        # print(f"   💾 Model saved.")

In [7]:
# ============================================================================
# EXECUTION
# ============================================================================
if __name__ == "__main__":
    # Initialize
    recommender = MovieRecommender(config)

    # Prepare & Train
    recommender.prepare_data()
    recommender.train()

    # Demonstration: Recommendations for a sample user
    sample_user = recommender.ratings_df['userId'].iloc[0]
    print(f"\n🎬 Generating recommendations for User ID: {sample_user}")

    recs = recommender.recommend_top_k(sample_user, k=5)

    if recs is not None:
        print(recs[['movieId', 'predicted_rating', 'title'] if 'title' in recs.columns else ['movieId', 'predicted_rating']].to_string(index=False))

🚀 Initializing on device: cuda
📂 Loading data from /content/drive/MyDrive/dataset/ratings.csv...
   ✓ Loaded metadata for 62423 movies
   ✓ Users: 162,541 | Items: 59,047 | Ratings: 25,000,095
   ✓ Data split & Loaders ready (Batch Size: 2048)

🔄 Starting Training...


Epoch 1/10 [Train]: 100%|██████████| 9766/9766 [04:55<00:00, 33.06it/s, loss=0.763]


   Done. Train Loss: 0.7833 | Test Loss: 0.7656 | RMSE: 0.8750


Epoch 2/10 [Train]: 100%|██████████| 9766/9766 [04:49<00:00, 33.71it/s, loss=0.705]


   Done. Train Loss: 0.7633 | Test Loss: 0.7628 | RMSE: 0.8734


...

   Done. Train Loss: 0.7619 | Test Loss: 0.7622 | RMSE: 0.8731


Epoch 9/10 [Train]: 100%|██████████| 9766/9766 [04:35<00:00, 35.41it/s, loss=0.741]


   Done. Train Loss: 0.7620 | Test Loss: 0.7625 | RMSE: 0.8732


Epoch 10/10 [Train]: 100%|██████████| 9766/9766 [04:41<00:00, 34.67it/s, loss=0.789]


   Done. Train Loss: 0.7620 | Test Loss: 0.7615 | RMSE: 0.8726

🎬 Generating recommendations for User ID: 1
 movieId  predicted_rating   title
     318          4.423855 Unknown
  159817          4.407789 Unknown
    2019          4.353211 Unknown
     858          4.316284 Unknown
  170705          4.306252 Unknown
